In [1]:
import numpy as np
import copy 

In [2]:
a = 1 ; a_ = 21 
b = 3 ; b_ = 13
gamma = 0.9

In [3]:
#Matrix Population

#Actions
# 0 = up
# 1 = left
# 2 = down
# 3 = right

#Rewards
#0 = 0
#1 = -1
#2 = 10 Special case of a
#3 = 5 Special case of b
P = np.zeros((25, 4, 25, 4)) #p(s', r|s, a) distribution
rc_size = 5
rewards = [0, -1, 10, 5]
for state in range(25):
    for action in range(4):
        if state == a:
            P[state][action][a_][2] += 1
        elif state == b:
            P[state][action][b_][3] += 1
        else:
            #other cases
            u_x = state//rc_size
            u_y = state%rc_size
            if action == 0:
                if u_x - 1 >= 0:
                    P[state][action][(u_x - 1) * rc_size + u_y][0] += 1
                else:
                    P[state][action][state][1] += 1
            elif action == 1:
                if u_y - 1 >= 0:
                    P[state][action][(u_x * rc_size) + (u_y - 1)][0] += 1
                else:
                    P[state][action][state][1] += 1
            elif action == 2:
                if u_x + 1 < rc_size:
                    P[state][action][(u_x + 1) * rc_size + u_y][0] += 1
                else:
                    P[state][action][state][1] += 1
            elif action == 3:
                if u_y + 1 < rc_size:
                    P[state][action][(u_x * rc_size) + (u_y + 1)][0] += 1
                else:
                    P[state][action][state][1] += 1
                    

In [14]:
#Code for Optimal Values and Policy Generation using Policy Iteration Method

def next_lookup1(Values, state, gamma):
    V_actions = np.zeros(4)
    for action in range(len(V_actions)):
        for otherstate in range(rc_size*rc_size):
            for reward in range(len(rewards)):
                V_actions[action] += P[state][action][otherstate][reward]*(rewards[reward] + gamma*Values[otherstate])
    
    return V_actions
    

def policy_evaluation(Values, Policy, gamma = 1.0, theta = 0.1):
    while True:
        Updated = np.zeros(rc_size * rc_size)
        delta = 0
        for state in range(0, rc_size*rc_size, 1):
            b_value = 0
            for action in range(4):
                for otherstate in range(rc_size*rc_size):
                    for reward in range(len(rewards)):
                        b_value += Policy[state][action] * P[state][action][otherstate][reward]*(rewards[reward] + gamma * Values[otherstate])
                
            delta = max(delta, np.abs(Values[state] - b_value))
            Updated[state]= b_value
        for state in range(rc_size*rc_size):
            Values[state] = Updated[state]
        if delta <= theta:
            break
    #print(count)
    return Values


def Policy_Iteration(gamma = 1.0, theta = 0.1):
    policy_state = True
    Values = np.zeros(rc_size * rc_size)
    Policy = np.zeros((rc_size * rc_size, 4)) + 0.25
    while True:
        policy_state = True
        Values = policy_evaluation(Values, Policy, gamma, theta)
        for state in range(0, rc_size * rc_size , 1):
            old_action = copy.deepcopy(Policy[state])
            V_actions = next_lookup1(Values, state, gamma)
            m = np.max(V_actions)
            count = np.count_nonzero(V_actions == m)
            updated_p = 1/count
            for i in range(len(V_actions)):
                if V_actions[i] == m:
                    Policy[state][i] = updated_p
                else:
                    Policy[state][i] = 0
            if np.array_equal(old_action,  Policy[state]) == False:
                policy_state = False
        if policy_state == True:
            break
    return Values, Policy
                
def getReadablePolicies(Policies):
    Policy = []
    for i in range(len(Policies)):
        s = ""
        for j in range(4):
            if j == 0 and Policies[i][j] > 0:
                s = s + "U"
            if j == 1 and Policies[i][j] > 0:
                s = s + "L"
            if j == 2 and Policies[i][j] > 0:
                s = s + "D"
            if j == 3 and Policies[i][j] > 0:
                s = s + "R"
        Policy.append(s)
    return Policy

In [15]:
Values, Policy = Policy_Iteration(0.9, 0.001)

In [16]:
print()
print(np.around(Values.reshape(5,5), decimals = 1))
print()
print(getReadablePolicies(Policy))




[[22.  24.4 22.  19.4 17.5]
 [19.8 22.  19.8 17.8 16. ]
 [17.8 19.8 17.8 16.  14.4]
 [16.  17.8 16.  14.4 13. ]
 [14.4 16.  14.4 13.  11.7]]

['R', 'ULDR', 'L', 'ULDR', 'L', 'UR', 'U', 'UL', 'L', 'L', 'UR', 'U', 'UL', 'UL', 'UL', 'UR', 'U', 'UL', 'UL', 'UL', 'UR', 'U', 'UL', 'UL', 'UL']


In [17]:
# functions for value iteration 
rewards = [0, -1, 10, 5]
def next_lookup(Values, state, gamma):
    V_actions = np.zeros(4)
    for action in range(len(V_actions)):
        for otherstate in range(rc_size*rc_size):
            for reward in range(len(rewards)):
                V_actions[action] += P[state][action][otherstate][reward]*(rewards[reward] + gamma*Values[otherstate])
    #print(V_actions)
    return V_actions/4
    

def Value_Iteration(gamma = 1.0, theta = 0.1):
    count = 0
    Values = np.zeros(rc_size * rc_size)
    Updated = np.zeros(rc_size * rc_size)
    while True:
        count += 1
        delta = 0
        for state in range(1, rc_size*rc_size-1, 1):
            b_value = np.max(next_lookup(Values, state, gamma))
            delta = max(delta, Values[state] - b_value)
            Values[state]= b_value
        #for state in range(rc_size*rc_size):
         #   Values[state] = Updated[state]
        #print(Values.reshape(4,4))
        if delta <= theta:
            break
    print(count)
    return Values

def getPolicy(Values, p, gamma = 0.9):
    Policy = np.zeros((rc_size*rc_size, 4)) 
    Policy[1:len(Policy)-1] += 0.25
    for state in range(rc_size*rc_size):
        V_actions = next_lookup(Values, state, gamma)
        m = np.max(V_actions)
        count = np.count_nonzero(V_actions == m)
        updated_p = 1/count
        for i in range(len(V_actions)):
            if V_actions[i] == m:
                Policy[state][i] = updated_p
            else:
                Policy[state][i] = 0
    return Policy

def getReadablePolicies(Policies):
    Policy = []
    for i in range(len(Policies)):
        s = ""
        for j in range(4):
            if j == 0 and Policies[i][j] > 0:
                s = s + "U"
            if j == 1 and Policies[i][j] > 0:
                s = s + "L"
            if j == 2 and Policies[i][j] > 0:
                s = s + "D"
            if j == 3 and Policies[i][j] > 0:
                s = s + "R"
        Policy.append(s)
    return Policy

def policy_evaluation1(Policy, gamma = 1.0, theta = 0.1):
    Values = np.zeros(rc_size * rc_size) 
    for i in range(100):
        for state in range(0, rc_size*rc_size, 1):
            b_value = 0
            for action in range(4):
                for otherstate in range(rc_size*rc_size):
                    for reward in range(len(rewards)):
                        b_value += Policy[state][action] * P[state][action][otherstate][reward]*(rewards[reward] + gamma * Values[otherstate])
                
            #delta = max(delta, np.abs(Values[state] - b_value))
            Values[state]= b_value

    return Values
      

In [18]:
Values = Value_Iteration(1, 0.001)
#print(np.around(Values.reshape((5,5)), decimals = 1))
Po = getPolicy(Values, P)
print(getReadablePolicies(Po))
print(np.around(policy_evaluation1(Po, 0.9, 0.001).reshape(5,5), decimals = 1))



1
['R', 'ULDR', 'L', 'ULDR', 'L', 'R', 'U', 'UL', 'U', 'UL', 'R', 'U', 'UL', 'U', 'UL', 'R', 'U', 'UL', 'U', 'UL', 'R', 'U', 'UL', 'U', 'UL']
[[22.  24.4 22.  18.5 16.6]
 [19.8 22.  19.8 16.6 14.9]
 [17.8 19.8 17.8 14.9 13.5]
 [16.  17.8 16.  13.5 12.1]
 [14.4 16.  14.4 12.1 10.9]]
